In [ ]:
### NOTEBOOK 2: 02_merge_and_clean.ipynb ###
#
# GOAL: Memuat file "failsafe" yang sudah di-geocode, membersihkan
#       bug 'description' Platform B, menggabungkannya, dan menjalankan
#       feature engineering (Zipcode Fix & Waterfall).
#
# INPUT: platform_a_geocoded.csv, platform_b_geocoded.csv
# OUTPUT: master_cleaned_features.csv
#

import pandas as pd
import numpy as np
import json
import re
import time
import os
import sys
from pathlib import Path
from tqdm import tqdm
from thefuzz import fuzz # Dibutuhkan untuk Zipcode Fix
import warnings

# Suppress warnings
warnings.filterwarnings('ignore')
pd.options.mode.chained_assignment = None
tqdm.pandas()

print("--- 02_merge_and_clean.ipynb ---")

In [ ]:
# ## Step 1: File Paths
# ---

print("Step 1: Setting up file paths...")

# Path Definitions
PROJECT_ROOT = Path(r"..")
DATA_DIR = PROJECT_ROOT / "data"
PROCESSED_DIR = DATA_DIR / "processed"
NOTEBOOK_DIR = PROJECT_ROOT / "notebooks" # Lokasi notebook ini

# --- INPUT FILES (Failsafes dari Notebook 1) ---
R123_GEOCODED_PATH = PROCESSED_DIR / "platform_a_geocoded.csv"
PLATFORM_B_GEOCODED_PATH = PROCESSED_DIR / "platform_b_geocoded.csv"

# --- OUTPUT FILE (Failsafe baru) ---
CLEANED_MASTER_PATH = PROCESSED_DIR / "master_cleaned_features.csv"

print(f"Input 1: {R123_GEOCODED_PATH}")
print(f"Input 2: {PLATFORM_B_GEOCODED_PATH}")
print(f"Output: {CLEANED_MASTER_PATH}\n")

In [ ]:
# ---
# ## Step 2: Load, Fix, and Merge
# ---

print("Step 2: Loading, Fixing, and Merging Data...")

try:
    # Tentukan tipe data untuk kolom yang bermasalah saat load
    r123_dtypes = {'zipcode': 'str', 'geo_confidence': 'str'}
    df_r123 = pd.read_csv(R123_GEOCODED_PATH, dtype=r123_dtypes)
    print(f"Loaded {len(df_r123)} records from platform_a_geocoded.csv")
except FileNotFoundError:
    print(f"❌ ERROR: File not found at {R123_GEOCODED_PATH}. Aborting.")
    sys.exit()
except Exception as e:
    print(f"❌ ERROR loading {R123_GEOCODED_PATH}: {e}")
    sys.exit()

try:
    # Tentukan tipe data untuk kolom yang bermasalah saat load
    ninetynineco_dtypes = {'zipcode': 'str', 'geo_confidence': 'str', 'description': 'str'}
    df_platform_b = pd.read_csv(PLATFORM_B_GEOCODED_PATH, dtype=ninetynineco_dtypes)
    print(f"Loaded {len(df_platform_b)} records from platform_b_geocoded.csv")
except FileNotFoundError:
    print(f"❌ ERROR: File not found at {PLATFORM_B_GEOCODED_PATH}. Aborting.")
    sys.exit()
except Exception as e:
    print(f"❌ ERROR loading {PLATFORM_B_GEOCODED_PATH}: {e}")
    sys.exit()


# --- *** CRITICAL FIX: Membersihkan Kolom Deskripsi Platform B *** ---
if 'description' in df_platform_b.columns and 'description_clean' in df_platform_b.columns:
    print("Fixing Platform B description columns (dropping 'description', renaming 'description_clean')...")
    df_platform_b = df_platform_b.drop(columns=['description'])
    df_platform_b = df_platform_b.rename(columns={'description_clean': 'description'})
else:
    print("Warning: Kolom 'description' dan 'description_clean' tidak ditemukan, mungkin sudah diperbaiki.")
# ----------------------------------------------------------------

# --- Grand Merge ---
print("Merging Platform B and Platform A dataframes...")
df_master = pd.concat([df_platform_b, df_r123], ignore_index=True)

# Buat satu kolom alamat master untuk pencocokan
df_master['master_address'] = df_master['geo_address'].fillna(df_master['address']).astype(str)

print(f"Successfully merged. Total listings: {len(df_master)}\n")

In [ ]:
# ---
# ## Step 3: Unified Zipcode Fix (Logika Anda dari IPYNB #8)
# ---

print("---")
print("## Step 3: Unified Zipcode Fix")
print("---")

def find_best_match_hierarchical(target_row, source_df):
    """
    Menemukan pencocokan zipcode terbaik menggunakan lokasi fuzzy dan harga sebagai tie-breaker.
    """
    target_location = target_row['master_address']
    target_price = target_row['price']

    # 1. Temukan pencocokan lokasi terbaik
    location_scores = source_df['master_address'].apply(
        lambda source_loc: fuzz.token_sort_ratio(target_location, str(source_loc))
    )
    max_loc_score = location_scores.max()
    
    # Filter hanya ke pencocokan lokasi terbaik
    best_location_matches = source_df[location_scores == max_loc_score]

    # 2. Gunakan Harga sebagai tie-breaker
    if len(best_location_matches) == 1:
        best_match = best_location_matches.iloc[0]
    else:
        # Pastikan harga adalah numerik untuk perbandingan
        best_location_matches['price'] = pd.to_numeric(best_location_matches['price'], errors='coerce')
        price_differences = (best_location_matches['price'] - target_price).abs()
        best_match = best_location_matches.loc[price_differences.idxmin()]
        
    return pd.Series({
        'zipcode_fuzzy': best_match['zipcode'],
        'zipcode_match_score': max_loc_score
    })

def fix_missing_zipcodes(df):
    """Menerapkan logika pencocokan fuzzy untuk mengisi NaN di 'zipcode'."""
    print("Running Unified Zipcode Fix...")
    df['price'] = pd.to_numeric(df['price'], errors='coerce')
    
    # Konversi 'zipcode' ke string untuk konsistensi, ubah NaNs menjadi placeholder
    df['zipcode'] = df['zipcode'].astype(str).replace('nan', np.nan)

    source_df = df.dropna(subset=['zipcode', 'master_address', 'price']).copy()
    target_df = df[df['zipcode'].isna() & df['master_address'].notna() & df['price'].notna()].copy()
    
    if target_df.empty:
        print("No missing zipcodes to fix. Skipping.")
        return df
        
    print(f"Applying fuzzy match to fix {len(target_df)} missing zipcodes...")
    
    match_results = target_df.progress_apply(
        lambda row: find_best_match_hierarchical(row, source_df),
        axis=1
    )
    
    df = df.join(match_results)
    
    # Isi kolom 'zipcode' asli dengan yang dari fuzzy-matched
    df['zipcode'] = df['zipcode'].fillna(df['zipcode_fuzzy'])
    
    print("Fuzzy matching for zipcodes complete.\n")
    return df

# Jalankan fungsinya
df_master = fix_missing_zipcodes(df_master)

Filling Missing Values

"Waterfall" Logic  
For each row in the master data, I run these 3 checks:

1. Check Main Column (e.g., bathrooms)  
First, I check if the bathrooms column already has a value.  
If Yes: (For example, the scraper Platform A already provided 3.0). I keep this value. The process for this row is finished.  
If No: (The value is NaN or empty), the data "falls" to the next step.  

2. Parse specs Column (JSON)  
Next, I call the parse_specs function. This function looks inside the specs column (which contains a JSON string such as {"bathrooms": "2", "land area": "100 m²", ...}).  
If Yes: I search for the aliases I defined (such as bathrooms, bath, or km). If found, I extract the value (e.g., "2"), clean it into the number 2, and use it to fill the empty bathrooms column. Process finished.  
If No: (The JSON does not contain a bathroom alias or the specs column itself is empty), the data "falls" to the last step.  

3. Parse description Column (Text)  
Finally, I call the parse_description function. This function uses Regular Expressions (Regex) to "read" the description text (for example, "Nice house in Setiabudi... 2 bath... 3 bed...").  
If Yes: I search for the patterns I defined (such as (\d+)\s*bath or (\d+)\s*bathrooms). If found, I extract the number (e.g., 2 from "2 bath"), and use it to fill the bathrooms column.  
If No: (No pattern found in the text), I give up.  


In [ ]:
# ---
# ## Step 4: "Waterfall" Feature Engineering (Logika Anda dari IPYNB #9)
# ---

print("---")
print("## Step 4: 'Waterfall' Feature Engineering")
print("---")

def clean_value(value_str):
    try:
        # Hapus 'm²' atau 'm' lalu ambil angkanya
        value_str = str(value_str).lower().replace('m²', '').replace('m', '').strip()
        match = re.search(r'([\d\.]+)', value_str) # Ambil angka, bisa jadi float
        if match: 
            return int(float(match.group(0))) # Ubah ke float dulu, lalu int
    except (TypeError, ValueError, AttributeError): pass
    return np.nan

def parse_specs(specs_str, aliases):
    try:
        specs_dict = json.loads(str(specs_str).lower().replace("'", '"'))
        specs_keys_lower = {k.lower(): v for k, v in specs_dict.items()}
        for alias in aliases:
            if alias in specs_keys_lower:
                return clean_value(specs_keys_lower[alias])
    except (json.JSONDecodeError, TypeError, AttributeError): pass
    return np.nan

def parse_description(description_str, patterns):
    try:
        text = str(description_str).lower()
        for pattern in patterns:
            match = re.search(pattern, text)
            if match:
                for group in match.groups():
                    if group: return clean_value(group)
    except (TypeError, AttributeError): pass
    return np.nan

def fill_data_waterfall(df, column, specs_aliases, desc_patterns):
    # Pastikan kolom target ada
    if column not in df.columns: 
        df[column] = np.nan
        
    df[column] = pd.to_numeric(df[column], errors='coerce')
    initial_missing = df[column].isna().sum()
    if initial_missing == 0:
        print(f"-> No missing values for '{column}'. Skipping.")
        return df

    def waterfall_filler(row):
        # 1. Cek nilai yang sudah ada
        if pd.notna(row[column]): return row[column]
        # 2. Parse 'specs'
        value = parse_specs(row['specs'], specs_aliases)
        if pd.notna(value): return value
        # 3. Parse 'description' (sekarang sudah bersih)
        value = parse_description(row['description'], desc_patterns)
        if pd.notna(value): return value
        return np.nan

    print(f"Filling {initial_missing} missing values for '{column}'...")
    df[column] = df.progress_apply(waterfall_filler, axis=1)
    filled_count = initial_missing - df[column].isna().sum()
    print(f"-> Filled {filled_count} missing values for '{column}'.")
    return df

# Tentukan fitur dan aliasnya
feature_cols = ['bedrooms', 'bathrooms', 'land_size_sqm', 'building_size_sqm']
all_aliases = {
    'bedrooms': (['kamar tidur', 'kt', 'bedrooms'], [r'(\d+)\s*kt', r'(\d+)\s*kamar tidur']),
    'bathrooms': (['kamar mandi', 'km', 'bathrooms'], [r'(\d+)\s*km', r'(\d+)\s*kamar mandi']),
    'land_size_sqm': (['luas tanah', 'lt', 'land size', 'luas lahan'], [r'lt\s*(\d+)', r'luas tanah\s*(\d+)']),
    'building_size_sqm': (['luas bangunan', 'lb', 'building size'], [r'lb\s*(\d+)', r'luas bangunan\s*(\d+)'])
}

# Jalankan waterfall untuk setiap fitur
for col in feature_cols:
    aliases, patterns = all_aliases[col]
    df_master = fill_data_waterfall(df_master, col, aliases, patterns)

print("Waterfall feature engineering complete.\n")

In [ ]:
print("---")
print("## Step 5: Saving Failsafe File")
print("---")

try:
    # Tipe data akhir sebelum menyimpan
    final_numeric_cols = ['price', 'bedrooms', 'bathrooms', 'land_size_sqm', 'building_size_sqm', 'latitude', 'longitude']
    for col in final_numeric_cols:
        if col in df_master.columns:
            df_master[col] = pd.to_numeric(df_master[col], errors='coerce')
            
    # Konversi kolom Int yang bisa nullable
    for col in ['bedrooms', 'bathrooms', 'land_size_sqm', 'building_size_sqm']:
         if col in df_master.columns:
            df_master[col] = df_master[col].astype('Int64')

    df_master.to_csv(CLEANED_MASTER_PATH, index=False, encoding='utf-8-sig')
    
    print(f"\n✅✅✅ 02_merge_and_clean.ipynb COMPLETE! ✅✅✅")
    print(f"New failsafe file saved to:")
    print(f"{CLEANED_MASTER_PATH}")
    print(f"\nTotal listings in file: {len(df_master)}")
    print("Ready for Notebook 3 (Deduplication).")
except Exception as e:
    print(f"❌ ERROR: Failed to save file. {e}")